In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train_users_2.csv')
test = pd.read_csv('test_users.csv')

In [3]:
print train.shape
print test.shape

(213451, 16)
(62096, 15)


### Equalize features in dataframes to be combined

In [4]:
[i for i in train.columns.values if i not in test.columns.values]

['country_destination']

In [5]:
y_train = train[[i for i in train.columns.values if i not in test.columns.values]]

In [6]:
# train.drop([i for i in train.columns.values if i not in test.columns.values], inplace=True, axis = 1)

In [7]:
combined = train.append(test)

In [8]:
combined.shape

(275547, 16)

In [9]:
### age_x_test and age_x_train represent the set to give me ages

### with ids they will be merged later back into train and test originals

In [10]:
age_x_test = combined[combined.age.isnull()] 

In [11]:
age_x_test.shape

(116866, 16)

In [12]:
age_x_train = combined[combined.age.notnull()] 

In [13]:
age_x_train.shape

(158681, 16)

In [14]:
age_target = age_x_train.age

In [15]:
age_x_train.head()

,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,date_first_booking,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,timestamp_first_active
1,seo,google,38.0,NDF,2011-05-25,NaN,untracked,Chrome,Mac Desktop,MALE,820tgsjxq7,en,Web,0,facebook,20090523174809
2,direct,direct,56.0,US,2010-09-28,2010-08-02,untracked,IE,Windows Desktop,FEMALE,4ft3gnwmtx,en,Web,3,basic,20090609231247
3,direct,direct,42.0,other,2011-12-05,2012-09-08,untracked,Firefox,Mac Desktop,FEMALE,bjjt8pjhuk,en,Web,0,facebook,20091031060129
4,direct,direct,41.0,US,2010-09-14,2010-02-18,untracked,Chrome,Mac Desktop,-unknown-,87mebub9p4,en,Web,0,basic,20091208061105
6,other,craigslist,46.0,US,2010-01-02,2010-01-05,untracked,Safari,Mac Desktop,FEMALE,lsw9q7uk0j,en,Web,0,basic,20100102012558


In [16]:
# pd.to_datetime(age_x_train.timestamp_first_active.map(lambda x: str(x)[0:8]))

In [17]:
all_ = age_x_train[['gender', 'signup_method', 'language',
                            'signup_app', 'first_device_type' ]]
allx = pd.get_dummies(all_)

In [18]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(allx, age_target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [19]:
all_x = age_x_test[['gender', 'signup_method', 'language',
                            'signup_app', 'first_device_type' ]]
allxtest = pd.get_dummies(all_x)

In [20]:
print allx.shape
print allxtest.shape

(158681, 46)
(116866, 46)


In [21]:
[i for i in allxtest.columns.values if i not in allx.columns.values]

['language_-unknown-']

In [22]:
pd.Series(model.predict(allxtest)).map(lambda x: x if x < 75 else 75).map(lambda x: x if x > 18 else 18).unique()

array([ 28.,  24.,  61.,  32.,  30.,  36.,  26.,  34.,  31.,  75.,  33.,
        21.,  44.,  45.,  29.,  39.,  49.,  22.,  42.,  25.,  54.,  27.,
        41.,  37.,  35.,  40.,  47.,  46.,  59.,  19.,  23.,  20.,  55.,
        67.,  38.,  53.,  65.,  50.,  18.,  66.,  51.,  52.,  43.])

In [29]:
age_x_test.loc[:,'age'] = pd.Series(model.predict(allxtest)).map(lambda x: x if x < 75 else 75).map(lambda x: x if x > 18 else 18)

In [30]:
age_x_test.age

0        28.0
5        61.0
12       61.0
16       32.0
18       61.0
20       32.0
23       24.0
26       36.0
38       30.0
43       32.0
44       24.0
46       61.0
47       32.0
48       24.0
51       61.0
57       61.0
59       34.0
60       31.0
62       24.0
63       24.0
66       61.0
67       30.0
68       61.0
69       30.0
70       24.0
73       75.0
78       61.0
80       61.0
82       32.0
83       61.0
         ... 
62035    24.0
62037    30.0
62038    61.0
62041    24.0
62045    29.0
62047    24.0
62048    32.0
62050    32.0
62052    34.0
62053    24.0
62061    24.0
62062    30.0
62063    32.0
62064    24.0
62065    24.0
62066    32.0
62068    32.0
62071    32.0
62072    32.0
62073    32.0
62074    24.0
62077    32.0
62078    29.0
62082    29.0
62086    24.0
62090    32.0
62092    28.0
62093    24.0
62094    32.0
age       NaN
Name: age, dtype: float64

In [31]:
train.shape

(213451, 16)

In [32]:
a_train = pd.merge( left=train, right= age_x_test , on= 'id', how = 'inner' )

In [33]:
a1_train = pd.merge( left=train, right=age_x_train, on= 'id', how = 'inner' )

In [35]:
train_merge = pd.concat([a_train, a1_train], axis=0)

In [36]:
train_merge.drop([i for i in train_merge.columns if '_x' in i], axis=1, inplace = True)

In [37]:
train_merge.columns = train_merge.columns.str.replace('_y', '')
train_merge.columns

Index([u'id', u'affiliate_channel', u'affiliate_provider', u'age',
       u'country_destination', u'date_account_created', u'date_first_booking',
       u'first_affiliate_tracked', u'first_browser', u'first_device_type',
       u'gender', u'language', u'signup_app', u'signup_flow', u'signup_method',
       u'timestamp_first_active'],
      dtype='object')

In [38]:
print train.shape 
print train_merge.shape

(213451, 16)
(213451, 16)


In [39]:
train_merge.to_pickle('train_age')